In [ ]:
# ===========================
# FINAL UPDATED CODE VERSION
# Precision & Recall Target > 0.60
# ===========================

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.metrics import (
    classification_report, precision_recall_curve,
    precision_score, recall_score
)
from xgboost import XGBClassifier
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline
import joblib

In [ ]:
# =========================================================
# 1. LOAD DATA
# =========================================================
df = pd.read_csv('bank-additional-full.csv', sep=';')

In [ ]:
display(df.head())

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


In [ ]:
df['target'] = df['y'].apply(lambda x: 1 if x == 'yes' else 0)
X = df.drop(['y', 'target', 'duration'], axis=1)
y = df['target']

In [ ]:
# Ubah label 'y' menjadi target numerik
df['target'] = df['y'].apply(lambda x: 1 if x == 'yes' else 0)

# Hapus kolom 'duration' secara permanen dari DataFrame
df.drop(['duration'], axis=1, inplace=True)

# Pisahkan fitur (X) dan label (y)
X = df.drop(['y', 'target'], axis=1)
y = df['target']


In [ ]:
display(df.head())

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y,target,fatigued_client,economic_pressure,never_contacted,deposit_influencer
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,93.994,-36.4,4.857,5191.0,no,0,0,456.528858,1,0
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,93.994,-36.4,4.857,5191.0,no,0,0,456.528858,1,0
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,93.994,-36.4,4.857,5191.0,no,0,0,456.528858,1,0
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,93.994,-36.4,4.857,5191.0,no,0,0,456.528858,1,0
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,93.994,-36.4,4.857,5191.0,no,0,0,456.528858,1,0


In [ ]:
print('Berikut adalah jumlah baris dan kolomnya:')
df.shape

Berikut adalah jumlah baris dan kolomnya:


(41188, 21)

In [ ]:
print("Nama-nama kolom di DataFrame:")
print(df.columns)
display(df.head())

Nama-nama kolom di DataFrame:
Index(['age', 'job', 'marital', 'education', 'default', 'housing', 'loan',
       'contact', 'month', 'day_of_week', 'campaign', 'pdays', 'previous',
       'poutcome', 'emp.var.rate', 'cons.price.idx', 'cons.conf.idx',
       'euribor3m', 'nr.employed', 'y', 'target'],
      dtype='object')


,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y,target
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no,0
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no,0
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no,0
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no,0
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no,0


Error: Runtime no longer has a reference to this dataframe, please re-run this cell and try again.


In [ ]:
# =========================================================
# 2. FEATURE ENGINEERING
# =========================================================
def apply_custom_features(df_input):
    df_eng = df_input.copy()
    df_eng['fatigued_client'] = (df_eng['campaign'] > 4).astype(int)
    df_eng['economic_pressure'] = df_eng['euribor3m'] * df_eng['cons.price.idx']
    df_eng['never_contacted'] = (df_eng['pdays'] == 999).astype(int)

    # Bonus fitur interaksi kuat
    df_eng['deposit_influencer'] = (
        ((df_eng['euribor3m'] < 2).astype(int)) &
        (df_eng['poutcome'] == 'success').astype(int)
    )

    return df_eng

df = apply_custom_features(df)
X = df.drop(['target', 'y'], axis=1)
y = df['target']

In [ ]:
# =========================================================
# 3. SPLIT DATA
# =========================================================
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

In [ ]:
# =========================================================
# 4. PIPELINE PREPROCESSING
# =========================================================
numeric_features = X.select_dtypes(include=['int64', 'float64']).columns
categorical_features = X.select_dtypes(include=['object']).columns

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ]
)

In [ ]:

# =========================================================
# 5. MODEL SETUP + SMOTE
# =========================================================
ratio = float(np.sum(y_train == 0)) / np.sum(y_train == 1)
sqrt_ratio = np.sqrt(ratio)

xgb = XGBClassifier(
    use_label_encoder=False,
    eval_metric='logloss',
    objective='binary:logistic',
    random_state=42
)

pipeline = ImbPipeline([
    ('preprocessor', preprocessor),
    ('smote', SMOTE(random_state=42)),
    ('classifier', xgb)
])

# Hyperparameter tuning fokus precision + recall
param_dist = {
    'classifier__n_estimators': [300, 500],
    'classifier__max_depth': [4, 6, 8],
    'classifier__learning_rate': [0.01, 0.05, 0.1],
    'classifier__gamma': [1, 3, 5],
    'classifier__min_child_weight': [3, 5, 7],
    'classifier__subsample': [0.6, 0.8],
    'classifier__colsample_bytree': [0.6, 0.8],
    'classifier__scale_pos_weight': [ratio, sqrt_ratio]
}

random_search = RandomizedSearchCV(
    pipeline,
    param_distributions=param_dist,
    n_iter=15,
    scoring='f1',
    cv=3,
    verbose=2,
    random_state=42,
    n_jobs=-1
)

print("\n🔄 Hyperparameter tuning dimulai...")
random_search.fit(X_train, y_train)

print("\n✅ Parameter terbaik ditemukan:")
print(random_search.best_params_)

best_model = random_search.best_estimator_


🔄 Hyperparameter tuning dimulai...
Fitting 3 folds for each of 15 candidates, totalling 45 fits


/usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [06:41:31] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)



✅ Parameter terbaik ditemukan:
{'classifier__subsample': 0.6, 'classifier__scale_pos_weight': np.float64(2.806531022331723), 'classifier__n_estimators': 500, 'classifier__min_child_weight': 7, 'classifier__max_depth': 4, 'classifier__learning_rate': 0.1, 'classifier__gamma': 5, 'classifier__colsample_bytree': 0.6}


In [ ]:
# =========================================================
# 6. FIND THRESHOLD (TARGET PRECISION & RECALL >= 0.60)
# =========================================================
y_prob = best_model.predict_proba(X_test)[:, 1]
precisions, recalls, thresholds = precision_recall_curve(y_test, y_prob)

best_threshold = None

for th in np.arange(0.1, 0.9, 0.01):
    y_pred = (y_prob >= th).astype(int)
    p = precision_score(y_test, y_pred)
    r = recall_score(y_test, y_pred)
    if p >= 0.60 and r >= 0.60:
        best_threshold = th
        print(f"\n🎯 TARGET TERCAPAI pada threshold={th:.2f}: precision={p:.3f}, recall={r:.3f}")
        break

if best_threshold is None:
    best_threshold = thresholds[np.argmax(precisions * recalls)]
    print(f"\n⚠️ Target 0.6 belum tercapai, gunakan threshold optimal F1: {best_threshold:.4f}")



⚠️ Target 0.6 belum tercapai, gunakan threshold optimal F1: 0.4970


In [ ]:
# =========================================================
# 7. FINAL EVALUATION
# =========================================================
y_pred_final = (y_prob >= best_threshold).astype(int)

print("\n--- FINAL CLASSIFICATION REPORT ---")
print(classification_report(y_test, y_pred_final))


--- FINAL CLASSIFICATION REPORT ---
              precision    recall  f1-score   support

           0       0.95      0.93      0.94      7310
           1       0.51      0.57      0.54       928

    accuracy                           0.89      8238
   macro avg       0.73      0.75      0.74      8238
weighted avg       0.90      0.89      0.89      8238



In [ ]:
# =========================================================
# 8. SAVE MODEL
# =========================================================
joblib.dump(best_model, "bank_xgb_tuned.joblib")
print("\n💾 Model berhasil disimpan: bank_xgb_tuned.joblib")


💾 Model berhasil disimpan: bank_xgb_tuned.joblib
